In [11]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import numpy as np
import pandas as pd
from glob import glob
import os
import statsmodels.api as sm
import itertools
from tqdm import tqdm
import calpgs
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


# Meta data

In [12]:
DATA_DIR = "../compile-data/out/per-trait-info/"

COVAR_COLS = ["AGE", "SEX", "DEPRIVATION_INDEX"] + [f"PC{i}" for i in range(1, 11)]

DATA_URL = "../r2-diff/data-table.xlsx"

df_trait_info = pd.read_excel(DATA_URL, sheet_name=0)
trait_map = {
    row.id: row.short if row.short is not np.nan else row.description
    for _, row in df_trait_info.iterrows()
}

df_covar_info = pd.read_excel(DATA_URL, sheet_name=1)
covar_map = {row.id: row.short for _, row in df_covar_info.iterrows()}

df_display = pd.read_excel(DATA_URL, sheet_name=2)

# Build calibration model

In [17]:
def build_model(data_prefix: str, ci_adjust: str, out_prefix: str):
    assert ci_adjust in ["none", "all"]

    df_train = pd.read_csv(data_prefix + ".train.tsv", sep="\t", index_col=0)
    df_test = pd.read_csv(data_prefix + ".test.tsv", sep="\t", index_col=0)

    # train model
    if ci_adjust == "none":
        ci_adjust_vars = None
    elif ci_adjust == "all":
        ci_adjust_vars = df_train.iloc[:, 3:]
    else:
        raise NotImplementedError

    # calibrate
    model = calpgs.calibrate_model(
        y=df_train["pheno"].values,
        pred=df_train["pred"].values,
        predstd=df_train["predstd"].values,
        ci_method="scale",
        ci_adjust_vars=ci_adjust_vars,
    )

    # adjust model
    if ci_adjust == "none":
        ci_adjust_vars = None
    elif ci_adjust == "all":
        ci_adjust_vars = df_test.iloc[:, 3:]
    else:
        raise NotImplementedError

    df_test["cal_pred"], df_test["cal_predstd"] = calpgs.calibrate_adjust(
        model=model,
        pred=df_test["pred"].values,
        predstd=df_test["predstd"].values,
        ci_adjust_vars=ci_adjust_vars,
    )

    out_dir = os.path.dirname(out_prefix)
    os.makedirs(out_dir, exist_ok=True)

    with open(out_prefix + ".model", "wb") as f:
        pickle.dump(model, f)
    df_test.to_csv(out_prefix + ".test_info.tsv", sep="\t")

In [18]:
data_prefix_list = np.unique([p.split(".")[0] for p in glob("out/data/*/*")])
df_params = pd.DataFrame(
    [params for params in itertools.product(data_prefix_list, ["all", "none"])],
    columns=["data_prefix", "ci_adjust"],
)
df_params["out_prefix"] = (
    df_params.data_prefix.str.replace("/data/", "/model/")
    + "-"
    + df_params["ci_adjust"]
)

In [19]:
for _, param in tqdm(df_params.iterrows(), total=len(df_params)):
    build_model(
        data_prefix=param.data_prefix,
        ci_adjust=param.ci_adjust,
        out_prefix=param.out_prefix,
    )

 72%|███████▎  | 58/80 [02:03<00:56,  2.55s/it]/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
 98%|█████████▊| 78/80 [03:07<00:04,  2.22s/it]/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
100%|██████████| 80/80 [03:22<00:00,  2.53s/it]
